In [29]:
from PIL import Image
from matplotlib.pyplot import imshow
from glob import glob
from os.path import splitext
import numpy as np
%matplotlib inline
import math
import os
import csv
from scipy import fftpack

def colorlayout(im_1):
    #Image partitioning
    #We divided the picture into M*N 64 block and the float is ignored 
    width,height=im_1.size
    im = im_1.crop((0,0,(width/8)*8,(height/8)*8))
    pixel=im.load()
    width,height = im.size
    #some parameter
    bw=width/8
    bh=height/8
    DCT_x=[]
    Y=[]
    Cr=[]
    Cb=[]
    #Representative color
    for w in xrange(0,width,bw):
        for h in xrange(0,height,bh):
            #the block number(w,h) width has width/8 and the height has height/8
            avgycrcb=[0,0,0] #the avg color of the block
            for i in range(bw):
                for j in range(bh):
                    #read the Y,Cr,Cb from pixel(w+i,h+j)
                    y,k,r=pixel[w+i,h+j]
                    avgycrcb=[aa+bb for aa, bb in zip(avgycrcb,[y,k,r])]
            avgycrcb=[aa/bb for aa, bb in zip(avgycrcb,[bw*bh,bw*bh,bw*bh])]
            DCT_x.extend([avgycrcb])
            Y.extend([avgycrcb[0]])
            Cr.extend([avgycrcb[2]])
            Cb.extend([avgycrcb[1]])
        
            #set the color
            for i in range(bw):
                for j in range(bh):
                    #set the avgrgb to pixel(w+i,h+j)
                    pixel[w+i,h+j]=(avgycrcb[0],avgycrcb[1],avgycrcb[2])

    #find the parameter of DCT
    BpqY=[]
    BpqCr=[]
    BpqCb=[]
    M=8
    N=8
    for p in xrange(M):
        for q in xrange(N):    
            aphp=math.sqrt(2/8.0)
            aphq=math.sqrt(2/8.0)
            if(p==0):
                aphp=1/(math.sqrt(M))
            if(q==0):
                aphq=1/(math.sqrt(N))
            result_Y=0.0
            result_Cr=0.0
            result_Cb=0.0
            for m in xrange(8):
                for n in xrange(8):
                    AmnY=Y[m*8+n]
                    AmnCr=Cr[m*8+n]
                    AmnCb=Cb[m*8+n]
                    result_Y+=AmnY*math.cos((math.pi*(2*m+1)*p)/(2*M))*math.cos((math.pi*(2*n+1)*q)/(2*N))
                    result_Cr+=AmnCr*math.cos((math.pi*(2*m+1)*p)/(2*M))*math.cos((math.pi*(2*n+1)*q)/(2*N))
                    result_Cb+=AmnCb*math.cos((math.pi*(2*m+1)*p)/(2*M))*math.cos((math.pi*(2*n+1)*q)/(2*N))
            result_Y=aphp*aphq*result_Y
            result_Cr=aphp*aphq*result_Cr
            result_Cb=aphp*aphq*result_Cb
            BpqY.insert(p*8+q,result_Y)
            BpqCr.insert(p*8+q,result_Cr)
            BpqCb.insert(p*8+q,result_Cb)
    #Bpq is the parameter of the DCT 
    #the parameter of the zigzag scan
    right=True
    sly_right=False
    sly_left=True
    down=False
    zzY=[]
    zzCr=[]
    zzCb=[]
    now=0
    times=1
    loc=0
    #zigzag scanned
    zzY.insert(loc,BpqY[now])#1
    zzCr.insert(loc,BpqCr[now])
    zzCb.insert(loc,BpqCb[now])
    revese=True
    while(now !=63):
        if(revese):
            now+=1
            loc+=1
        else:
            now+=8
            loc+=1
        zzY.insert(loc,BpqY[now])#2,4,6,8,24,40,56
        zzCr.insert(loc,BpqCr[now])
        zzCb.insert(loc,BpqCb[now])
        for i in xrange(times):
            now+=7
            loc+=1
            zzY.insert(loc,BpqY[now])#9,11,18,25,13,20,27,34,41,15,22,29,36,43,50,57,31,38,45,52,59,47,54,61,63
            zzCr.insert(loc,BpqCr[now])
            zzCb.insert(loc,BpqCb[now])
        if(times<7 and revese):
            times+=1#times=2,4,6
            now+=8
            loc+=1
        else:
            revese=False
            now+=1
            loc+=1
            times-=1 #times=6,4,2,0
        zzY.insert(loc,BpqY[now])#17,33,49,58,60,62
        zzCr.insert(loc,BpqCr[now])
        zzCb.insert(loc,BpqCb[now])
        if(times !=0):
            for i in xrange(times):
                now-=7
                loc+=1
                zzY.insert(loc,BpqY[now])#10,3,26,19,12,5,42,35,28,21,14,7,51,44,37,30,23,16,53,46,39,32,55,48
                zzCr.insert(loc,BpqCr[now])
                zzCb.insert(loc,BpqCb[now])
        if(times<7 and revese):
            times+=1#times=3,5,7
        else:
            times-=1#times=5,3,1
    return (zzY,zzCb,zzCr)


top10dis=[] #show the most like distance
imlist=[] #show the img which is the top 10 
for i in range(10):
    top10dis.insert(i,float('inf'))
    
im_1=Image.open('dataset/1.jpg').convert('YCbCr')
[Y_1,Cb_1,Cr_1]=colorlayout(im_1)
top10dis=[] #show the most like distance
imlist=[] #show the img which is the top 10 
for i in xrange(10):
    top10dis.insert(i,float('inf'))

wyi=1.0
wcri=0.5
wcbi=0.5
for i in xrange(1006):
    if(not(i==220 or i==472)):
        now_im=Image.open('dataset/'+str(i+1)+'.jpg').convert('YCbCr')
        [Y_n,Cb_n,Cr_n]=colorlayout(now_im)
    dis_y=0.0
    dis_cb=0.0
    dis_cr=0.0
    for j in xrange(64):
        dis_y+=((Y_1[j]-Y_n[j])**2)*wyi
        dis_cb+=((Cb_1[j]-Cb_n[j])**2)*wcri
        dis_cr+=((Cr_1[j]-Cr_n[j])**2)*wcbi
    dis=math.sqrt(dis_y)+math.sqrt(dis_cb)+math.sqrt(dis_cr)
    for k in range(10):
        if(dis<top10dis[k]):
            top10dis.insert(10,dis)
            imlist.insert(k,str(i+1)+'.jpg')
            top10dis.sort()
            del top10dis[10]
            if(len(imlist)>10):
                del imlist[10]
            break
#print the result
print '(distance,image_no,rank)'
for i in xrange(10):
    print '('+ str(top10dis[i])+','+str(imlist[i])+','+str(i+1)+')'
    
    

(distance,image_no,rank)
(0.0,1.jpg,1)
(218.479374624,102.jpg,2)
(243.457718824,86.jpg,3)
(269.640790735,415.jpg,4)
(293.341839424,737.jpg,5)
(327.015167209,16.jpg,6)
(334.746076277,427.jpg,7)
(335.624703064,9.jpg,8)
(340.715654671,206.jpg,9)
(353.166376786,387.jpg,10)


In [48]:


DATASET = "dataset_update"
color_layout_data = list()
for filename in os.listdir(DATASET):
    
    img = Image.open(DATASET+'/'+filename).convert('YCbCr')

    [Y_n,Cb_n,Cr_n] = colorlayout(img)
    
    img_row =[filename,','.join(repr(i) for i in Y_n),','.join(repr(i) for i in Cb_n),','.join(repr(i) for i in Cr_n)]
    color_layout_data.append(img_row)
    f = open("color_layout_update.csv","w")
    w = csv.writer(f)
    w.writerows(color_layout_data)
    f.close()
    
        

In [49]:
q_img = Image.open('dataset_update/ukbench00000.jpg').convert('YCbCr')
[Y_1,Cb_1,Cr_1] = colorlayout(q_img)

wyi=1.0
wcri=0.5
wcbi=0.5

f = open('color_layout_update.csv','r')
rank = list()
count = 0
for row in csv.reader(f):
    
    Y_n = [float(y) for y in row[1].split(',')]
    Cb_n = [float(cb) for cb in row[2].split(',')]
    Cr_n = [float(cr) for cr in row[3].split(',')]
    
    dis,dis_y,dis_cb,dis_cr = 0.0,0.0,0.0,0.0
    
    for j in xrange(64):
        dis_y+=((Y_1[j]-Y_n[j])**2)*wyi
        dis_cb+=((Cb_1[j]-Cb_n[j])**2)*wcri
        dis_cr+=((Cr_1[j]-Cr_n[j])**2)*wcbi
    dis=math.sqrt(dis_y)+math.sqrt(dis_cb)+math.sqrt(dis_cr)
    #dis = dis_y**0.5+dis_cb**0.5+dis_cr**0.5
    rank_row = (row[0],dis)
    rank.append(rank_row)
f.close()

#print rank

rank = sorted(rank,key=lambda rank:rank[1])
print rank[0:10]

[('ukbench00000.jpg', 0.0), ('ukbench00101.jpg', 218.47937462429564), ('ukbench00085.jpg', 243.45771882360012), ('ukbench00414.jpg', 269.64079073485425), ('ukbench00736.jpg', 293.3418394243112), ('ukbench00015.jpg', 327.01516720937707), ('ukbench00426.jpg', 334.74607627714425), ('ukbench00008.jpg', 335.6247030642667), ('ukbench00205.jpg', 340.7156546707024), ('ukbench00386.jpg', 353.1663767864274)]


In [42]:
now_im=Image.open('dataset_update/ukbench00000.jpg').convert('YCbCr')
[Y_n,Cb_n,Cr_n]=colorlayout(now_im)

Y_n,Cb_n,Cr_n = colorlayout(now_im)


print Y_n
print Cb_n
print Cr_n


[1209.8749999999998, -6.174438288626283, -6.9755200424865125, 253.08689059514114, -5.859835250042124, 549.933600531312, -31.649830607194545, 11.655649927487218, 12.302222801836223, -0.6253715951103944, 142.12499999999994, 13.034888099960291, -117.74056178736114, 4.530113108509184, 141.37499999999974, 46.917330762546285, -6.3471760871045575, -32.22442522568653, -11.818029513881008, 52.450852003626636, 10.29248948662308, 82.82772518234941, -3.7695539466074166, -87.28985962519573, -9.01697859600447, -86.50972797906012, 2.4536545743738927, -55.01310077306529, 32.322455173977524, -3.5220955892193793, 19.076001966252832, 10.921269996133226, -49.491795336467376, -3.8390583034842436, 26.836786064147834, 6.610838878135884, 0.18771681881723445, -52.293601141128846, 4.755373443345324, -30.874999999999957, -7.767597329663843, -23.54360114112867, 3.164746045808082, 21.168310189047485, 9.506907529895766, -15.027939985287112, -7.811051327270139, -22.68275636745644, -3.923596352506843, 2.3851801904578